See [metric_learning.ipynb](metric_learning.ipynb) for how the neighbors are generated.

In [46]:
import pickle

import numpy as np

with open('/data/patch_features_small/test2014_positive.p', 'rb') as f:
    eval_data = pickle.load(f)
    categories = eval_data['categories']
    labels = np.array([label.data.numpy() for label in eval_data['labels']])
    
with open('/data/patch_features_small/test2014_neighbors.p', 'rb') as f:
    neighbor_distances, neighbor_indices = pickle.load(f)

with open('/data/patch_features_small/train2014_embeddings.p', 'rb') as f:
    embeddings_dict = pickle.load(f)
    neighbor_labels = embeddings_dict['labels']
    
eval_data_index = np.sum(labels, axis=1) == 1
labels = np.array([np.argwhere(label).reshape(-1) for label in labels[eval_data_index]])
neighbor_distances = neighbor_distances[eval_data_index]
neighbor_indices = neighbor_indices[eval_data_index]

AttributeError: 'tuple' object has no attribute 'reshape'

In [98]:
K = 10
categories

[{'supercategory': 'vehicle', 'id': 2, 'name': 'bicycle'},
 {'supercategory': 'vehicle', 'id': 3, 'name': 'car'},
 {'supercategory': 'vehicle', 'id': 4, 'name': 'motorcycle'},
 {'supercategory': 'vehicle', 'id': 5, 'name': 'airplane'},
 {'supercategory': 'vehicle', 'id': 6, 'name': 'bus'},
 {'supercategory': 'vehicle', 'id': 7, 'name': 'train'},
 {'supercategory': 'vehicle', 'id': 8, 'name': 'truck'},
 {'supercategory': 'vehicle', 'id': 9, 'name': 'boat'},
 {'supercategory': 'animal', 'id': 16, 'name': 'bird'},
 {'supercategory': 'animal', 'id': 17, 'name': 'cat'},
 {'supercategory': 'animal', 'id': 18, 'name': 'dog'},
 {'supercategory': 'animal', 'id': 19, 'name': 'horse'},
 {'supercategory': 'animal', 'id': 20, 'name': 'sheep'},
 {'supercategory': 'animal', 'id': 21, 'name': 'cow'},
 {'supercategory': 'animal', 'id': 22, 'name': 'elephant'},
 {'supercategory': 'animal', 'id': 23, 'name': 'bear'},
 {'supercategory': 'animal', 'id': 24, 'name': 'zebra'},
 {'supercategory': 'animal', 'i

In [105]:
from sklearn.metrics import average_precision_score

def compute_average_precision_score(k: int, category_index: int = -1) -> float:
    in_class_pct = np.array([
        np.sum(neighbor_labels[neighbor_indices[i, :k]], axis=0)[label]
        for i, label in enumerate(labels)
    ])
    if category_index > -1:
        in_class_pct = in_class_pct[labels == category_index]
    return np.mean(in_class_pct/k)

[compute_average_precision_score(3, idx) for idx in range(18)]
#for k in range(5,10):
#    print(k)

[0.044669677,
 0.21247225,
 0.04137072,
 0.033873733,
 0.06308611,
 0.018129377,
 0.113515936,
 0.049425654,
 0.10515619,
 0.0677447,
 0.07159624,
 0.0815782,
 0.14295326,
 0.14960404,
 0.07540984,
 0.013939395,
 0.11569622,
 0.07865787]

In [93]:

[np.mean(np.mean(neighbor_distances[:,:k + 1], axis=1)) for k in range(K)]

[2.638774508746533,
 2.693671808395852,
 2.7377323932265845,
 2.774306352025918,
 2.8066751064634192,
 2.8356251240947694,
 2.86200062866835,
 2.886508929904211,
 2.9094412984201954,
 2.930872725544902]

In [10]:
neighbors_distance[eval_data_index]

array([[4.41345988, 4.41345988, 4.77582108, ..., 5.36257026, 5.36257026,
        5.36257026],
       [3.94142012, 4.51828671, 4.51828671, ..., 4.6726689 , 4.6726689 ,
        4.73340961],
       [5.57449322, 5.57449322, 5.7560028 , ..., 6.48144006, 6.48144006,
        6.48144006],
       ...,
       [2.69425411, 2.91350494, 2.91350494, ..., 2.93062071, 2.93062071,
        2.93062071],
       [2.72859023, 2.87309512, 3.1241912 , ..., 3.16238074, 3.16238074,
        3.16238074],
       [3.96354994, 3.96354994, 3.96354994, ..., 4.21241378, 4.21241378,
        4.21241378]])